In [1]:
#Add Dependencies
import pandas as pd
import numpy as np
import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### Connection to the Postgres database and Retrive data from the tables

In [2]:
#Read Data from the PostgressSql
conn = pg.connect(user = "postgres",
                                  password = "akshaj10",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "movie_data")
sql1 = "Select * From movie_metadata;"
sql2 = "Select * from award;"
movie_df = pd.read_sql_query(sql1, conn)
award_df = pd.read_sql_query(sql2, conn)
conn = None

In [3]:
#Load Data into Pandas Data frame and Display the dataframe.
movie_df.shape

(45006, 13)

In [4]:
#Load Data into Pandas Data frame and Display the dataframe.
award_df.shape

(10395, 8)

In [5]:
#Merge the two dataframe
combined_df = pd.merge(movie_df, award_df,how='left', left_on='title', right_on='film' )
combined_df.shape

(51598, 21)

In [6]:
# Filter all rows for US as the country
movie_us_df = combined_df[combined_df['country'] == 'United States of America'] 
movie_us_df.shape

(22651, 21)

In [7]:
# Find null values
for column in movie_us_df.columns:
    print(f"Column{column} has {movie_us_df[column].isnull().sum()} null values")

Columnimdb_id has 3 null values
Columnbudget has 0 null values
Columnoriginal_title has 0 null values
Columnpopularity has 0 null values
Columnrelease_date has 2 null values
Columnrevenue has 0 null values
Columnruntime has 5 null values
Columntitle has 0 null values
Columnvote_average has 0 null values
Columnvote_count has 0 null values
Columnproduction has 2317 null values
Columncountry has 0 null values
Columnnew_genre has 0 null values
Columnindex has 15911 null values
Columnyear_film has 15911 null values
Columnyear_ceremony has 15911 null values
Columnceremony has 15911 null values
Columncategory has 15911 null values
Columnname has 15911 null values
Columnfilm has 15911 null values
Columnwinner has 15911 null values


In [8]:
#Set the index to IMDB_ID
movie_us_df = movie_us_df.set_index("imdb_id")

In [9]:
columns =['budget','original_title','popularity','revenue','runtime','vote_average','vote_count','production','country','new_genre','category','year_film', 'winner']

target = ['winner']

In [10]:
movie_us_df = movie_us_df.loc[:, columns].copy()
movie_us_df.shape

(22651, 13)

In [11]:
#Convert winner column into numbers 
movie_us_df["winner"] = movie_us_df["winner"].replace({True: 1, False: 0})
movie_us_df.shape

(22651, 13)

In [12]:
movie_us_df['budget']=movie_us_df.budget.astype('int64')

In [13]:
# Create a title Dataframe.
title_df=pd.DataFrame(data=movie_us_df['original_title'],index=movie_us_df.index,copy=True)
title_df.head()

,original_title
imdb_id,
tt0114709,Toy Story
tt0114709,Toy Story
tt0114709,Toy Story
tt0113497,Jumanji
tt0113228,Grumpier Old Men


In [14]:
# Remove the movie title from the dataframe
movie_us_df = movie_us_df.drop('original_title',1)
movie_us_df.head()

,budget,popularity,revenue,runtime,vote_average,vote_count,production,country,new_genre,category,year_film,winner
imdb_id,,,,,,,,,,,,
tt0114709,30000000,21.946943,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,MUSIC (Original Musical or Comedy Score),1995.0,0.0
tt0114709,30000000,21.946943,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,MUSIC (Original Song),1995.0,0.0
tt0114709,30000000,21.946943,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,WRITING (Screenplay Written Directly for the S...,1995.0,0.0
tt0113497,65000000,17.015539,262797249.0,104.0,6.9,2413.0,TriStar Pictures,United States of America,Adventure,NaN,NaN,NaN
tt0113228,0,11.7129,0.0,101.0,6.5,92.0,Warner Bros.,United States of America,Romance,NaN,NaN,NaN


In [15]:
# Generate our categorical variable list for movie dataset
movie_cat = movie_us_df.dtypes[movie_us_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
movie_us_df[movie_cat].nunique()

popularity    18127
production     4611
country           1
new_genre        20
category        100
dtype: int64

In [16]:
# Drop null rows
movie_us_df = movie_us_df.dropna(axis = 0,how= 'any')
movie_us_df.shape


(6597, 12)

In [17]:
# let's examine how many columns we will obtain after one hot encoding these variables
encode_df=pd.get_dummies(movie_us_df[movie_cat], drop_first=True)
encode_df.shape

(6597, 2947)

In [18]:
# Merge one-hot encoded features and drop the originals
oscar_us_df = movie_us_df.merge(encode_df,left_index=True, right_index=True)
oscar_us_df = oscar_us_df.drop(movie_cat,1)
oscar_us_df.head(2)

,budget,revenue,runtime,vote_average,vote_count,year_film,winner,popularity_0.00118,popularity_0.001191,popularity_0.001224,...,category_WRITING (Screenplay Written Directly for the Screen),category_WRITING (Screenplay Written Directly for the Screen--based on factual material or on story material not previously published or produced),category_WRITING (Screenplay),category_WRITING (Screenplay--Adapted),category_WRITING (Screenplay--Original),category_WRITING (Screenplay--based on material from another medium),category_WRITING (Story and Screenplay),category_WRITING (Story and Screenplay--based on factual material or material not previously published or produced),category_WRITING (Story and Screenplay--based on material not previously published or produced),category_WRITING (Story and Screenplay--written directly for the screen)
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0002101,0,0.0,88.0,4.0,1.0,1934.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tt0002101,0,0.0,88.0,4.0,1.0,1934.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Create a Logistic Regression Model

In [19]:
# Define the features set.
X = oscar_us_df.copy()
X = X.drop('winner', axis=1)
X.shape

(36263, 2953)

In [20]:
# Define the target set(output label)
y = oscar_us_df['winner']
y.head(2)

imdb_id
tt0002101    0.0
tt0002101    0.0
Name: winner, dtype: float64

In [21]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [22]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(27197, 2953)
(9066, 2953)
(27197,)
(9066,)


In [23]:
# Creating the logistic Regression classifier instance.
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=78)
# Fitting the model.
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=78, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
# Making predictions using the testing data.
y_pred = classifier.predict(X_test)
#results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
#results.head(20)

In [25]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

cm_df

,Predicted 0,Predicted 1
Actual 0,5877,811
Actual 1,2027,351


In [26]:
# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.686962276637988

In [27]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,5877,811
Actual 1,2027,351


Accuracy Score : 0.686962276637988
Classification Report
              precision    recall  f1-score   support

         0.0       0.74      0.88      0.81      6688
         1.0       0.30      0.15      0.20      2378

    accuracy                           0.69      9066
   macro avg       0.52      0.51      0.50      9066
weighted avg       0.63      0.69      0.65      9066

